In [1]:
import os
import sys

import numpy as np
import pandas as pd
import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="ticks", color_codes=True)

sys.path.append('../src/features')
import build_features as bf

In [2]:
originalDF = pd.read_csv("../data/raw/application_train.csv")

In [3]:
cleanTrain=bf.executeFeatures(originalDF)

## Build Model

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [5]:
X_train, X_test, y_train, y_test = train_test_split(cleanTrain.iloc[:,1:], 
                                                    cleanTrain.TARGET, test_size = 0.2, random_state = 0)

In [6]:
clf = linear_model.LogisticRegression().fit(X_train, y_train)

In [8]:
clf.score(X_test, y_test)

0.9210607612636782